# Lesson 1: Advanced RAG Pipeline

In [1]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 6aedc788-d6bb-48fd-8028-8b0c0019324b
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Basic RAG pipeline

In [4]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [5]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
query_engine = index.as_query_engine()

In [7]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

Develop a side hustle, ensure the project will help you grow technically, collaborate with good teammates, and consider if the project can be a stepping stone to larger projects.


## Evaluation setup using TruLens

In [8]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [9]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [10]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [11]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [12]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [13]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [14]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [15]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_9a4e2e9a4c56700ac32eef7b91cf650a,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, worki...",-,"{""record_id"": ""record_hash_9a4e2e9a4c56700ac32...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-03-10T07:29:35.634320"", ""...",2025-03-10T07:29:36.699777,1.0,1.00,1.0,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",1,2067,0.003125
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_8c236b61eba918aed8251e061cba1bdf,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_8c236b61eba918aed82...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-03-10T07:29:36.819558"", ""...",2025-03-10T07:29:38.061218,1.0,0.50,0.9,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,1,1680,0.002547
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e4b57812d5ec268ca998d84ab196dcbb,"""What is the importance of networking in AI?""","""Networking is crucial in AI as it helps indiv...",-,"{""record_id"": ""record_hash_e4b57812d5ec268ca99...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-03-10T07:29:38.185969"", ""...",2025-03-10T07:29:39.761006,1.0,0.50,0.9,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,1,1684,0.002556
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_6a790ad0885de2d18743d896733d8d81,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_6a790ad0885de2d1874...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-03-10T07:29:39.895495"", ""...",2025-03-10T07:29:40.813431,1.0,0.95,NaN,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,NaN,0,1634,0.002471
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e69a71b28711d7f0275c779ec65203ca,"""How can altruism be beneficial in building a ...","""Helping others during your career journey can...",-,"{""record_id"": ""record_hash_e69a71b28711d7f0275...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-03-10T07:29:40.952177"", ""...",2025-03-10T07:29:41.640626,1.0,NaN,NaN,[{'args': {'prompt': 'How can altruism be bene...,NaN,NaN,0,1610,0.002423


In [16]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at https://s172-29-33-56p38560.lab-aws-production.deeplearning.ai/ .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [17]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [18]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [19]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [20]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

To get started on a personal project in AI, you can begin by identifying a project that aligns with your career goals and interests. It's important to choose a project that is responsible, ethical, and beneficial to people. Once you have selected a project, you can follow a structured approach by scoping the project, defining its objectives, and outlining the steps needed for execution. As you work on the project, aim to grow in terms of the project's scope, complexity, and impact over time. This progression will help you build a portfolio that demonstrates your skill development in AI.


In [21]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [22]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a supportive community are the keys to building a career in AI.
How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to leverage the diverse skills and perspectives of their colleagues. Working collaboratively enables team members to combine their technical expertise, problem-solving abilities, and insights to tackle complex AI projects more effectively. Additionally, being surrounded by motivated and knowledgeable teammates can inspire individuals to continuously learn and improve, ultimately leading to better project outcomes.
What is the importance of networking in AI?
Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field. By connecting with professionals who have experience in AI, individuals can gain 

In [23]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,1.0,0.357143,0.49881,1.363636,0.000813


In [24]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-33-56p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [25]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [26]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [27]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: 9a099e2a-8de5-4afc-a5f3-a9335ff20805.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: e615fc7f-4b72-4ff7-84e2-be617e1fbfe2.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

Building a portfolio of AI projects involves showcasing a progression from simple to complex undertakings over time. It is important to be able to communicate your thinking effectively to demonstrate the value of your work and gain the trust of others. Identifying ideas that are worth working on is a crucial skill for an AI architect, and working on projects across various industries can help in gaining experience and building a diverse portfolio.


In [28]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [29]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f0a10210700 is calling an instrumented method <function BaseQueryEngine.query at 0x7f0b043ab640>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f0a8c2e3b80) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f0a10210700 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f0afb42ea70>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f0a8c2e3b80) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x7f0aa5f67010 is calling an instrumented method <function BaseRetriever.retrieve at 0x7f0b043aa9e0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7

> Merging 2 nodes into parent node.
> Parent node id: ec668b55-c9ee-4547-bb37-68f55baf2c12.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 5896fcd6-4d64-4ca2-bcd1-58c9302375ec.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f0a10210670 is calling an instrumented method <function CompactAndRefine.get_response at 0x7f0aff93b520>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f0a8c2e2c20) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f0a10210670 is calling an instrumented method <function Refine.get_response at 0x7f0afed732e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f0a8c2e2c20) using this function.
A new object of type <class 'llama_index.llm_predictor.base.LLMPredictor'> at 0x7f0aa489b980 is calling an instrumented method <function LLMPredictor.predict at 0x7f0b0c879120>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthes

What are the keys to building a career in AI?
The keys to building a career in AI include learning foundational technical skills, working on projects, finding a job, and being part of a community. Additionally, collaborating with others, influencing, and being influenced by others are critical aspects for success in AI.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f0a10210670 is calling an instrumented method <function Refine.get_response at 0x7f0afed732e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f0a8c2e2c20) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f0a10210700 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f0afb42ea70>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f0a8c2e3b80) using this function.


...

What are some common challenges in AI?
Some common challenges in AI include the highly iterative nature of AI projects, difficulties in project management due to uncertainties in time estimates for achieving target accuracy, and technical challenges that many individuals face when working on AI projects.
> Merging 3 nodes into parent node.
> Parent node id: 3af6cf64-e5c9-4b04-9383-cc2a7298cb90.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: 0e63fe05-278c-4a1b-85c3-54280c52077e.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f0a10210670 is calling an instrumented method <function Refine.get_response at 0x7f0afed732e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f0a8c2e2c20) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f0a10210700 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f0afb42ea70>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f0a8c2e3b80) using this function.


Is it normal to find parts of AI challenging?
It is normal to find parts of AI challenging.
> Merging 1 nodes into parent node.
> Parent node id: 2da226c8-3dfd-4d06-a755-1c8b01a493ff.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 6b17dd7c-44a1-4d56-adac-a57c2e1ed361.
> Parent node text: If you’re leaving 
a job, exit gracefully. Give your employer ample notice, give your full effort...

> Merging 1 nodes into parent node.
> Parent node id: d198b182-0ef6-4a0c-a99f-2f60f731174f.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right JobCHAPTER 8
JOBS

> Merging 1 nodes into parent node.
> Parent node id: c2866345-514c-41ee-b8e1-51eeafc85d50.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: a8b7fa28-cce7-4468-83fb-25954f886886.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f0a10210670 is calling an instrumented method <function Refine.get_response at 0x7f0afed732e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f0a8c2e2c20) using this function.


What is the right AI job for me?
The right AI job for you may depend on whether you are looking to switch roles, industries, or both. If you are seeking your first job in AI, transitioning either roles or industries individually may be more manageable than attempting both simultaneously.


In [30]:
tru.get_leaderboard(app_ids=[])

,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_id,,,,,
Automerging Query Engine,1.0,0.6625,0.764524,2.272727,0.000862


In [31]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-33-56p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>